In [2]:
import cv2
import numpy as np
camera = video_frame = cv2.VideoCapture("new soccer video.mp4")
out = cv2.VideoWriter('output.mp4',-1, 20.0, (1720,750)) #cv2.VideoWriter('video.avi',cv2.VideoWriter_fourcc(*'MPEG'),30,(width,height))

firstFrame = None
p=0
y_1=330
h_1=1070
x_1=200
w_1=1800
# loop over the frames of the video
while True:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    (grabbed, frame1) = camera.read()
    if frame1 is None:
        break
    frame = frame1[y_1:y_1+h_1, x_1:x_1+w_1]
    lower = [100, 100, 100]# for color range of animal start to end
    upper = [180, 200, 200]
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    framecopy=frame.copy()
    detectarea = cv2.inRange(framecopy, lower, upper)#finding between values of given function, result is white and black pixels
    
    
    (_,cnts, _) = cv2.findContours(detectarea.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#finding contours of conected component
    for c in cnts:
        if cv2.contourArea(c)>5000:
            
            epsilon = 0.001*cv2.arcLength(c,True)
            #It approximates a contour shape to another shape with less number of 
            # vertices depending upon the precision we specify.
            approx = cv2.approxPolyDP(c,epsilon,True)
            # shows the approximated curve for epsilon = 1% of arc length
            
            cv2.drawContours(frame, [approx], -1, (0, 255, 255), 3)
            # drawing contours on approx points values.
    
    
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 1))
#     detectarea = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    if firstFrame is None or p==12:
        p=0
        firstFrame = detectarea.copy()
        #rawCapture.truncate(0)
        continue
    p=p+1
    
    frameDelta = cv2.absdiff(firstFrame, detectarea)
    
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
    
    kernel = np.ones((3,3),np.uint8)
    thresh1 = cv2.erode(thresh,kernel,iterations = 2)   # erosion, dialation
    thresh2 = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))# horizantal line kernal
    
    thresh1 = cv2.dilate(thresh2, kernel, iterations=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 9))# vertical line kernal
    thresh1 = cv2.dilate(thresh1, kernel, iterations=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))# horizantal line kernal
    thresh1 = cv2.dilate(thresh1, kernel, iterations=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))# horizantal line kernal
    thresh1 = cv2.dilate(thresh1, kernel, iterations=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 9))# vertical line kernal
    thresh1 = cv2.dilate(thresh1, kernel, iterations=2)# dialation with kernal iteration 2
    
    thresh1 = cv2.dilate(thresh1, kernel, iterations=2)
    
    
    (_,cnts, _) = cv2.findContours(thresh1.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)#finding contours of conected component
    
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 5000:
            continue
                # if the contour is too small, ignore it
        if cv2.contourArea(c) > 40000:
            
            continue
        # compute the bounding box for the contour, draw it on the frame,
        #print(cv2.contourArea(c))# and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        if (x > 300):
            if (y<580):
                if x<1400:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    out.write(frame)
    
#     cv2.imshow('fram',thresh)
    cv2.imshow('fram',frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break
# cleanup the camera and close any open windows
camera.release()
out.release()
cv2.destroyAllWindows()